<a href="https://colab.research.google.com/github/cedamusk/AI-N-ML/blob/main/Quantum_intelligence_and_predicting_earthquakes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install obspy pennylane scikit-learn numpy pandas matplotlib

In [ ]:
import numpy as np
import pennylane as qml
import pandas as pd
from obspy.signal.trigger import classic_sta_lta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [ ]:
def generate_synthetic_data(n_samples=1000):
  np.random.seed(42)

  time=np.linspace(0, 100, n_samples)
  background=np.random.normal(0, 0.05, n_samples)

  earthquakes=np.zeros(n_samples)
  for i in range(3):
    pos=np.random.randint(200, n_samples-200)
    magnitude=np.random.uniform(0.5, 1.0)

    earthquakes[pos:pos+20]+=magnitude*np.exp(-np.linspace(0, 2, 20))

    earthquakes[pos+20:pos+70]+=magnitude*1.5*np.exp(-np.exp(-np.linspace(0, 3, 50)))

  signal=background+earthquakes
  return time, signal

In [ ]:
n_qubits=4
dev=qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def quantum_circuit(inputs):
  for i in range(n_qubits):
    qml.RY(inputs[i], wires=i)

  for i in range(n_qubits-1):
    qml.CNOT(wires=[i, i+1])

  return [qml.expval(qml.PauliZ(i))for i in range (n_qubits)]


In [ ]:
def prepare_data(time, signal, window_size=20):
  X, y=[], []
  for i in range(len(signal)-window_size-1):
    feature_window=signal[i:i+window_size]

    features=[
        np.mean(feature_window),
        np.std(feature_window),
        np.max(feature_window),
        np.min(feature_window)
    ]

    X.append(features)
    y.append(signal[i+window_size])
  return np.array(X), np.array(y)

In [ ]:
time, signal=generate_synthetic_data()

In [ ]:
X, y=prepare_data(signal)
scaler_X=StandardScaler()
scaler_y=StandardScaler()

X_scaled=scaler_x.fit_transform(X)
y_scaled=scaled_y.fit_transform(y.reshape(-1, 1)).flatten()

In [ ]:
train_size=int(0.8*len(X))
X_train, X_test=X_scaled[:train_size], X_scaled[train_size:]
y_train, y_test=y[:train_size], y[train_size:]

In [ ]:
def quantum_predict(X, scaler_y):
  predictions=[]
  for sample in X:
    quantum_features=quantum_circuit(sample)
    prediction=np.mean(quantum_features)
    predictions.append(prediction)
  predictions=np.array(predictions)
  predictions=scaler_y.inverse_transform(predictions.reshape(-1, 1)).flatten()
  return np.array(predictions)

In [ ]:
def baseline_predict(X):
  return np.array([np.mean(X)]*len(X))

In [ ]:
y_pred_quantum=quantum_predict(X_test)
y_pred_baseline=baseline_predict(y_test)

In [ ]:
mse_quantum=mean_squared_error(y_test, y_pred_quantum)
r2_quantum=r2_score(y_test, y_pred_quantum)
mse_baseline=mean_squared_error(y_test, y_pred_baseline)
r2_baseline=r2_score(y_test, y_pred_baseline)

In [ ]:
plt.figure(figsize=(15, 10))

In [ ]:
plt.subplot(2,1,1)
plt.plot(time, signal, label="Original Signal")
plt.title('Synthetic Earthquake Data')
plt.legend()

In [ ]:
plt.subplot(2,1,2)
plt.plot(y_test[:100], label="Actual", color='blue')
plt.plot(y_pred_quantum[:100], label=" Quantum Prediction", color='red')
plt.plot(y_pred_baseline[:100], label='Baseline Prediction', color='green', linestyle='--')
plt.title('Prediction results Comparison')
plt.legend()

In [ ]:
plt.tight_layout()
plt.show()

In [ ]:
print("Quantum Model Metrics:")
print(f"mean squared error:{mse_quantum:.4f}")
print(f"R^2 score:{r2_quantum:.4f}")
print("\nBaseline Model metrics:")
print("\Mean Squared Error:{mse_baseline:.4f}")
print(f"R^2 Score:{r2_baseline}")